# Referências da Documentação AWS:

- https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-examples.html

- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html

- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse_stream.html

# Dependências

In [ ]:
import boto3
import json
import os

from datetime import datetime, timezone

# Constantes

In [ ]:
# Pasta local para salvar respostas das chamadas AWS
OUTPUT_FOLDER = "respostas"

# ID do modelo Amazon Nova Lite (modelo de linguagem da AWS)
MODEL_ID = 'amazon.nova-lite-v1:0'

# ARN do prompt criado no AWS Prompt Manager
PROMPT_ARN = 'arn:aws:bedrock:us-east-1:123456789012:prompt/AABBCC' # Se quiser usar a versão `DRAFT` é só deixar sem id, se quise especificar uma id, basta coloca :id (ex. `:1`)

# Configuração de diretórios

In [ ]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Configurações do AWS BEDROCK

In [ ]:
# Inicializa o cliente do Bedrock Runtime para fazer chamadas à API
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Configuração da requisição

In [ ]:
# Histórico da conversa
conversation_history = []

# Primeira pergunta
conversation_history.append({
    "role": "user",
    "content": [{"text": "Qual é a capital do Brasil?"}]
})

# Configurações
inference_config = {"temperature": 0.5, "maxTokens": 400}

# Teste do método converse

In [ ]:
# Primeira chamada
response = bedrock_runtime_client.converse(
        modelId=MODEL_ID,
        messages=conversation_history,
        inferenceConfig=inference_config)    

In [ ]:
# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/converse_01.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

In [ ]:
# Adicionar resposta do modelo ao contexto
model_response = response['output']['message']['content'][0]['text']
print(model_response)

In [ ]:
# Veja estado atual da conversa
print(json.dumps(conversation_history, indent=2, ensure_ascii=False))

In [ ]:
# Adicionar a resposta ao histórico de conversa
conversation_history.append({
    "role": "assistant",
    "content": [{"text": model_response}]
})

In [ ]:
# Veja estado atual da conversa
print(json.dumps(conversation_history, indent=2, ensure_ascii=False))

In [ ]:
# Adicionar uma nova pergunta antes da segunda chamada
conversation_history.append({
    "role": "user", 
    "content": [{"text": "Me conte mais sobre essa cidade"}]
})

In [ ]:
# Confira o histórico
print(json.dumps(conversation_history, indent=2, ensure_ascii=False))

In [ ]:
# Segunda pergunta relacionada
response = bedrock_runtime_client.converse(
        modelId=MODEL_ID,
        messages=conversation_history,
        inferenceConfig=inference_config)    

# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/converse_02.json", 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4, default=str)

# Adicionar resposta do modelo ao contexto
model_response = response['output']['message']['content'][0]['text']
print(model_response)

# Confira uso de tokens
print(f"Uso de tokens: {response['usage']}")

# Exemplo com Prompt Manager

In [ ]:
# Variáveis para o prompt de agendamento da barbearia
prompt_variables = {
    'usuario_nome': {'text': 'Ana'},
    'filme_sinopse': {'text': 'Uma jovem princesa embarca em uma aventura mágica para salvar seu reino, descobrindo poderes especiais e fazendo novos amigos pelo caminho.'},    
}

In [ ]:
# Chamada usando Prompt Manager
# IMPORTANTE: Quando usamos um prompt do Prompt Manager, as configurações
# de inferência (temperature, maxTokens, etc.) foram definidas no próprio
# prompt através do console AWS, não podem ser sobrescritas em runtime.
# 
# ❌ ERRO: Usar inferenceConfig aqui resultará em ValidationException:
# "Conflict encountered. Overriding 'inferenceConfig' during runtime is not yet supported"
# 
# ✅ CORRETO: Omitir inferenceConfig e usar apenas promptVariables
response_prompt = bedrock_runtime_client.converse(
    modelId=PROMPT_ARN,
    promptVariables=prompt_variables
)

# Exibir resposta
message_text = response_prompt['output']['message']['content'][0]['text']
print("Mensagem gerada pelo Prompt Manager:")
print(message_text)

# Salvar resultado
with open(f"{OUTPUT_FOLDER}/converse_03.json", 'w', encoding='utf-8') as json_file:
    json.dump(response_prompt, json_file, ensure_ascii=False, indent=4, default=str)

# Teste do método converse_stream

In [ ]:
# Teste do método converse_stream
conversation_stream = [
    {
        "role": "user",
        "content": [{"text": "Resuma a história do Brasil em 4 parágrafos."}]
    }
]

response_stream = bedrock_runtime_client.converse_stream(
    modelId=MODEL_ID,
    messages=conversation_stream,
    inferenceConfig=inference_config
)

# Salvar os resultados em um arquivo JSON
with open(f"{OUTPUT_FOLDER}/converse_04.json", 'w', encoding='utf-8') as json_file:
    json.dump(response_stream, json_file, ensure_ascii=False, indent=4, default=str)

# Variável para acumular o texto completo da resposta streaming
texto_stream = ""

print("\nResposta em tempo real: ", end="", flush=True)

# Processa cada evento do stream de resposta
for event in response_stream['stream']:
    # Verifica se o evento contém texto da resposta
    if 'contentBlockDelta' in event:
        # Extrai o texto parcial deste chunk
        delta_text = event['contentBlockDelta']['delta']['text']
        # Exibe o texto imediatamente (efeito de digitação em tempo real)
        print(delta_text, end="", flush=True)
        # Acumula o texto para ter a resposta completa no final
        texto_stream += delta_text
    
    # Gera timestamp único para cada chunk (para debug/análise)
    timestamp_parcial = datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S%f')
    
    # Salva cada evento individual para análise posterior
    filename_parcial = os.path.join(OUTPUT_FOLDER, f"partial_{timestamp_parcial}.json")
    with open(filename_parcial, 'w', encoding='utf-8') as json_file:
        json.dump(event, json_file, ensure_ascii=False, indent=4, default=str)

print("\n\nTexto completo da resposta:")
print(texto_stream)     